In [1]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

import pandas as pd
import numpy as np

from PIL import Image
from resizeimage import resizeimage

from skimage.feature import hog
from skimage.color import rgb2grey

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from sklearn.metrics import roc_curve, auc

In [2]:
def Scalar(feature_matrix):
    # define standard scaler
    ss = StandardScaler()
    # run this on our feature matrix
    images_stand = ss.fit_transform(feature_matrix)

    pca = PCA(n_components=500)
    # use fit_transform to run PCA on our standardized matrix
    images_pca = ss.fit_transform(images_stand)
    # look at new shape
    print('PCA matrix shape is: ', images_pca.shape)
    X = pd.DataFrame(images_pca)
    
    return X

In [3]:
path="C:\\Users\\Maximillian\\Desktop\\Code Demo\\Example\\Occulsion\\Green"

os.chdir(path)

In [4]:
A=np.load("390VesselOcculuded.npy")
A.shape

(4996, 176400)

In [5]:
#Let the index of A be from 0 to A.shape[0]/2
B=A[0:2498]
B=Scalar(B)

PCA matrix shape is:  (2498, 176400)


In [6]:
#Let the index of A be from A.shape[0]/2 to A.shape[0]+1
C=A[2498:4997]
C=Scalar(C)

PCA matrix shape is:  (2498, 176400)


In [7]:
#Change path to be the location of the SVM pickle
os.chdir("C:\\Users\\Maximillian\\Desktop\\Code Demo\\Example\\Pickles")

In [8]:
from sklearn.externals import joblib

# Load the model from the file 
svm = joblib.load('GreenLinear.pkl') 

C:\Users\Maximillian\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [ ]:
#The following steps may take time

In [9]:
ProbsB=svm.predict_proba(B)
print('First Half Done')
ProbsC=svm.predict_proba(C)
print('Second Half Done')

First Half Done
Second Half Done


In [10]:
Q=np.concatenate((ProbsB,ProbsC),axis=0)
Q.shape

(4996, 2)

In [11]:
os.chdir(path)
np.save("390GreenLinearPDProb",Q)

In [ ]:
# The follow is a method for running the above code in a single cell with limited edits. Note it has not been fully developed.

In [ ]:
channel='Green'
kernel='RBF'
image='390'

def predictProbs():
    #Go to location of saved occulsion files
    path='C:\\Users\\Maximillian\\Desktop\\Code Demo\\Example\\Occulsion\\' #+channel
    os.chdir(path)
    
    #Open Occuled Image
    file_name=image+'VesselOcculuded.npy'
    A=np.load(file_name)
    return(A)
    
def finalize(half,final,A,channel, kernel,image):
    B=A[0:half]
    B=Scalar(B)
    
    C=A[half:final]
    C=Scalar(C)
    
    #Pickle Locations
    os.chdir('C:\\Users\\Maximillian\\Desktop\\Code Demo\\Example\\Occulsion')
    
    # Load the model from the file 
    pickle_name=channel+kernel+'.pkl'
    svm = joblib.load(pickle_name) 
    
    ProbsB=svm.predict_proba(B)
    print('391 is Done')
    ProbsC=svm.predict_proba(C)
    print('406 is Done')
    
    Q=np.concatenate((ProbsB,ProbsC),axis=0)
    Q.shape
    
    path='C:\\Users\\Maximillian\\Desktop\\Code Demo\\Example\\Occulsion' #+channel
    os.chdir(path)
    save_name=image+channel+kernel+'PDProb'
    np.save(save_name,Q)